# The Merry Movie Montage

Your objective is to find a set of three strings containing every permutation of the seven symbols 🎅, 🤶, 🦌, 🧝, 🎄, 🎁, and 🎀 as substrings, subject to the following conditions:

* Every permutation must be in at least one of the three strings.
* Each permutation beginning with 🎅🤶 must be in all three strings.
* Each string may have up to two wildcards 🌟, which will match any symbol in a permutation. No string of length seven containing more than one wildcard will count as a permutation.
* Your score is the length of the longest of the three strings. This is a minimization problem, so lower scores are better.

https://www.kaggle.com/c/santa-2021/data

In [19]:
import numpy as np 
import pandas as pd 
import math
import operator
import time
import random

# The Data Provided
Let's explore the data that we are given.
We have a list of all possible permutations of the symbols. There are 5,040 different combinations.

In [8]:
permutations = pd.read_csv('assets/santa-2021/permutations.csv')
print(permutations.head(122))
print(len(permutations))

    Permutation
0       🎅🤶🦌🧝🎄🎁🎀
1       🎅🤶🦌🧝🎄🎀🎁
2       🎅🤶🦌🧝🎁🎄🎀
3       🎅🤶🦌🧝🎁🎀🎄
4       🎅🤶🦌🧝🎀🎄🎁
..          ...
117     🎅🤶🎀🎁🧝🎄🦌
118     🎅🤶🎀🎁🎄🦌🧝
119     🎅🤶🎀🎁🎄🧝🦌
120     🎅🦌🤶🧝🎄🎁🎀
121     🎅🦌🤶🧝🎄🎀🎁

[122 rows x 1 columns]
5040


In [9]:
permutations.iloc[0]

Permutation    🎅🤶🦌🧝🎄🎁🎀
Name: 0, dtype: object

In [10]:
# the permutation at each position is a pandas series
type(permutations.iloc[0])

pandas.core.series.Series

In [11]:
# a series object with all elements in the first position
permutations.iloc[0][0]

'🎅🤶🦌🧝🎄🎁🎀'

In [12]:
# the series object contains a string in the first position
type(permutations.iloc[0][0])

str

In [13]:
# we can index into each string element
permutations.iloc[0][0][0]

'🎅'

In [14]:
# we can use equality operators for string comparisons
permutations.iloc[0][0][0] == permutations.iloc[0][0][0]

True

In [15]:
permutations.iloc[0][0][0] == permutations.iloc[0][0][1]

False

We are given a distance matrix that shows the number of character changes across all permutations

In [20]:
distance_matrix = pd.read_csv('assets/santa-2021/distance_matrix.csv')
distance_matrix.head()

,Permutation,🎅🤶🦌🧝🎄🎁🎀,🎅🤶🦌🧝🎄🎀🎁,🎅🤶🦌🧝🎁🎄🎀,🎅🤶🦌🧝🎁🎀🎄,🎅🤶🦌🧝🎀🎄🎁,🎅🤶🦌🧝🎀🎁🎄,🎅🤶🦌🎄🧝🎁🎀,🎅🤶🦌🎄🧝🎀🎁,🎅🤶🦌🎄🎁🧝🎀,...,🎀🎁🎄🦌🤶🎅🧝,🎀🎁🎄🦌🤶🧝🎅,🎀🎁🎄🦌🧝🎅🤶,🎀🎁🎄🦌🧝🤶🎅,🎀🎁🎄🧝🎅🤶🦌,🎀🎁🎄🧝🎅🦌🤶,🎀🎁🎄🧝🤶🎅🦌,🎀🎁🎄🧝🤶🦌🎅,🎀🎁🎄🧝🦌🎅🤶,🎀🎁🎄🧝🦌🤶🎅
0,🎅🤶🦌🧝🎄🎁🎀,0,7,7,7,7,7,7,7,7,...,6,6,6,6,6,6,6,6,6,6
1,🎅🤶🦌🧝🎄🎀🎁,7,0,7,7,7,7,7,7,7,...,5,5,5,5,5,5,5,5,5,5
2,🎅🤶🦌🧝🎁🎄🎀,7,7,0,7,7,7,7,7,7,...,6,6,6,6,6,6,6,6,6,6
3,🎅🤶🦌🧝🎁🎀🎄,7,7,7,0,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
4,🎅🤶🦌🧝🎀🎄🎁,7,7,7,7,0,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7


We are allowed to use two wildcards (🌟) in our string

In [21]:
wildcards = pd.read_csv('assets/santa-2021/wildcards.csv')
wildcards.head(8)

,Factor,Permutation
0,🌟🤶🦌🧝🎄🎁🎀,🎅🤶🦌🧝🎄🎁🎀
1,🎅🌟🦌🧝🎄🎁🎀,🎅🤶🦌🧝🎄🎁🎀
2,🎅🤶🌟🧝🎄🎁🎀,🎅🤶🦌🧝🎄🎁🎀
3,🎅🤶🦌🌟🎄🎁🎀,🎅🤶🦌🧝🎄🎁🎀
4,🎅🤶🦌🧝🌟🎁🎀,🎅🤶🦌🧝🎄🎁🎀
5,🎅🤶🦌🧝🎄🌟🎀,🎅🤶🦌🧝🎄🎁🎀
6,🎅🤶🦌🧝🎄🎁🌟,🎅🤶🦌🧝🎄🎁🎀
7,🌟🤶🦌🧝🎄🎀🎁,🎅🤶🦌🧝🎄🎀🎁


This is waht a submission should look like

In [22]:
sample_submission = pd.read_csv('assets/santa-2021/sample_submission.csv')
sample_submission

,schedule
0,🎅🤶🦌🧝🎄🎁🎀🎁🎄🧝🦌🎅🤶🦌🧝🎄🎀🎁🤶🎄🧝🦌🎀🎅🤶🦌🧝🎁🎄🎀🤶🎁🎄🧝🦌🎅🤶🦌🧝🎁🎀🎄🤶🧝🦌🎀...
1,🎅🤶🦌🧝🎄🎁🎀🎅🤶🦌🧝🎄🎀🎁🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🧝🎀🎄🎁🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🎄...
2,🎅🤶🦌🧝🎄🎁🎀🎅🤶🦌🧝🎄🎀🎁🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🧝🎀🎄🎁🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🎄...


In [28]:
# we should be able to do better than 6985 for a baseline
len(sample_submission.iloc[2][0])

6985

## A Simple Brute Force Approach (no wildcards)
Since there are only 5,040 permutation, we are going to begin by constructinge a simple brute force approach. \

### Part A: Two strings that contain all permutations starting with 🎅, 🤶

For every permutation that starts with 🎅, 🤶 add the permutation (from the remaining permutations to add) that results in the shortest string. Return the shortest overall string.

In [64]:
# since there are 5 spots after 🎅, 🤶 and we cannot repeat symbols
# we can calcualte the number of permutations for all permutations that start with 🎅, 🤶  with 5!
math.factorial(5)

120

In [33]:
permutations_list = permutations.values.tolist()
permutations_list = [item for sublist in permutations_list for item in sublist]
list_of_candidates = permutations_list[:math.factorial(5)]

In [44]:
part_a_candidates = list_of_candidates.copy()
part_a_permutations = list_of_candidates.copy()
# For each potential starting string generate a candidate output
start = time.time()
for i in range(0, len(part_a_candidates)):
    start_candidate_sequence = time.time()
    current_sequence = part_a_candidates[i]
    print(f"Processing Candidate #{i}: {current_sequence}")
    # candidate string must contain all of the permutations
    permutations_remaining = part_a_permutations.copy()
    permutations_remaining.remove(current_sequence)
    print(f"{len(permutations_remaining)} permutations to add")
    while len(permutations_remaining) > 0:
        if len(permutations_remaining) % 1000 == 0:
#             print(f"    current string length: {len(current_sequence)}")
            print(f"    Permutations remaining... {len(permutations_remaining)}")
        best_permutation = permutations_remaining[0]
        best_added_length = 7
        for permutation in permutations_remaining:
            # check if already in sequence
            if operator.contains(current_sequence, permutation):
                permutations_remaining.remove(permutation)
                continue
            added_length = get_added_length(current_sequence, permutation)
            if added_length < best_added_length:
                best_added_length = added_length
                best_permutation = permutation
        current_sequence = (current_sequence + best_permutation[-best_added_length:])
        permutations_remaining.remove(best_permutation)
    part_a_candidates[i] = current_sequence
    print(f"    Candidate processed!")
    print(f"    Total Permutation length: {len(current_sequence)}")
    # Test that all substrings are part of the lists
    print(f"    All permutations are included: {all(substring in current_sequence for substring in part_a_permutations)}")
    end_candidate_sequence = time.time()
    print(f"Time to process candidate: {round(end_candidate_sequence - start_candidate_sequence)} seconds\n")
end = time.time()
print(f"Total time to process 20 candidates: {round(end - start)/60} minutes")

Processing Candidate #0: 🎅🤶🦌🧝🎁🎄🎀
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #1: 🎅🤶🦌🧝🎁🎀🎄
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #2: 🎅🤶🦌🧝🎀🎄🎁
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #3: 🎅🤶🦌🧝🎀🎁🎄
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #4: 🎅🤶🦌🎄🧝🎁🎀
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #5: 🎅🤶🦌🎄🧝

    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #50: 🎅🤶🎄🦌🎀🧝🎁
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #51: 🎅🤶🎄🦌🎀🎁🧝
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #52: 🎅🤶🎄🧝🦌🎁🎀
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #53: 🎅🤶🎄🧝🦌🎀🎁
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #54: 🎅🤶🎄🧝🎁🦌🎀
117 permutations to add
    Candidate processed!


    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #91: 🎅🤶🎁🎀🧝🎄🦌
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #92: 🎅🤶🎁🎀🎄🦌🧝
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #93: 🎅🤶🎁🎀🎄🧝🦌
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #94: 🎅🤶🎀🦌🧝🎄🎁
117 permutations to add
    Candidate processed!
    Total Permutation length: 826
    All permutations are included: True
Time to process candidate: 0 seconds

Processing Candidate #95: 🎅🤶🎀🦌🧝🎁🎄
117 permutations to add
    Candidate processed!


Because we only have 120 combinations, this runs very quickly

In [45]:
# the shortest list of these 20 candidates is 5,913
# but it's only 1/6 of our potential candidates for submission
len(min(part_a_candidates, key=len))

826

In [47]:
# indexes 0, 6, and 8 have thes shortest strings
# Our score will be the largest, 5,950
results = [(len(candidate), part_a_candidates.index(candidate)) for candidate in part_a_candidates]
results.sort()
print(results)

[(826, 0), (826, 1), (826, 2), (826, 3), (826, 4), (826, 5), (826, 6), (826, 7), (826, 8), (826, 9), (826, 10), (826, 11), (826, 12), (826, 13), (826, 14), (826, 15), (826, 16), (826, 17), (826, 18), (826, 19), (826, 20), (826, 21), (826, 22), (826, 23), (826, 24), (826, 25), (826, 26), (826, 27), (826, 28), (826, 29), (826, 30), (826, 31), (826, 32), (826, 33), (826, 34), (826, 35), (826, 36), (826, 37), (826, 38), (826, 39), (826, 40), (826, 41), (826, 42), (826, 43), (826, 44), (826, 45), (826, 46), (826, 47), (826, 48), (826, 49), (826, 50), (826, 51), (826, 52), (826, 53), (826, 54), (826, 55), (826, 56), (826, 57), (826, 58), (826, 59), (826, 60), (826, 61), (826, 62), (826, 63), (826, 64), (826, 65), (826, 66), (826, 67), (826, 68), (826, 69), (826, 70), (826, 71), (826, 72), (826, 73), (826, 74), (826, 75), (826, 76), (826, 77), (826, 78), (826, 79), (826, 80), (826, 81), (826, 82), (826, 83), (826, 84), (826, 85), (826, 86), (826, 87), (826, 88), (826, 89), (826, 90), (826, 91

### Part B: A String that contains all substrings
Create a superstring that has all permutations but does not have the restriction of starting with 🎅, 🤶

In [50]:
# get all permutations
permutations_list = permutations.values.tolist()
# flatten out the permutations list to change from list of lists to list of strings
permutations_list = [item for sublist in permutations_list for item in sublist]

In [51]:
# get the number of characters added to sequence1 by overlapping sequence2
def get_added_length(sequence1, sequence2):
    for i in range(1, len(sequence2)):
        # best solution: ends with everything except the last character
        if sequence1.endswith(sequence2[:-i]):
            return i
    # otherwise return the entire length of the sequence; no overlap
    return len(sequence2)

**One run through**: comparing all of the available permutations to add to the candidate permutations (the starting permutation) and adding the one that results in the shortest total added length to the candidate string

In [52]:
i = 0
new_list = []
permutations_to_add = permutations_list
for current_sequence in permutations_list:
    # only print first 10 results
    if i < 10:
        print(f"First Candidate Sequence: {current_sequence}")
    best_permutation = None
    best_added_length = 7
    for permutation in permutations_to_add:
        # check if already in sequence
        if operator.contains(current_sequence, permutation):
            permutations_to_add.remove(permutation)
            continue
        added_length = get_added_length(current_sequence, permutation)
        if added_length < best_added_length:
            best_added_length = added_length
            best_permutation = permutation
    if i < 10:
        print(f"Best Permutation: {best_permutation}")
        print(f"String to append: {best_permutation[-best_added_length:]}")
        print(f"New Sequence: {current_sequence + best_permutation[-best_added_length:]}")
    new_list.append(current_sequence + best_permutation[-best_added_length:])
    permutations_to_add.remove(best_permutation)
    i += 1
        

First Candidate Sequence: 🎅🤶🦌🧝🎄🎁🎀
Best Permutation: 🤶🦌🧝🎄🎁🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🧝🎄🎁🎀🎅
First Candidate Sequence: 🎅🤶🦌🧝🎁🎄🎀
Best Permutation: 🤶🦌🧝🎁🎄🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🧝🎁🎄🎀🎅
First Candidate Sequence: 🎅🤶🦌🧝🎀🎄🎁
Best Permutation: 🤶🦌🧝🎀🎄🎁🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🧝🎀🎄🎁🎅
First Candidate Sequence: 🎅🤶🦌🎄🧝🎁🎀
Best Permutation: 🤶🦌🎄🧝🎁🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎄🧝🎁🎀🎅
First Candidate Sequence: 🎅🤶🦌🎄🎁🧝🎀
Best Permutation: 🤶🦌🎄🎁🧝🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎄🎁🧝🎀🎅
First Candidate Sequence: 🎅🤶🦌🎄🎀🧝🎁
Best Permutation: 🤶🦌🎄🎀🧝🎁🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎄🎀🧝🎁🎅
First Candidate Sequence: 🎅🤶🦌🎁🧝🎄🎀
Best Permutation: 🤶🦌🎁🧝🎄🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎁🧝🎄🎀🎅
First Candidate Sequence: 🎅🤶🦌🎁🎄🧝🎀
Best Permutation: 🤶🦌🎁🎄🧝🎀🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎁🎄🧝🎀🎅
First Candidate Sequence: 🎅🤶🦌🎁🎀🧝🎄
Best Permutation: 🤶🦌🎁🎀🧝🎄🎅
String to append: 🎅
New Sequence: 🎅🤶🦌🎁🎀🧝🎄🎅
First Candidate Sequence: 🎅🤶🦌🎀🧝🎄🎁
Best Permutation: 🤶🦌🎀🧝🎄🎁🎅
String to app

A loop over a random starting candidats. This approach is simple, but would take a very long time to find the optimal solution.

In [59]:
permutations_list = permutations.values.tolist()
permutations_list = [item for sublist in permutations_list for item in sublist]

In [63]:
list_of_candidates = permutations_list.copy()
# For each potential starting string generate a candidate output
start = time.time()
for i in range(0, 50):
    i = random.randrange(0, len(permutations_list))
    start_candidate_sequence = time.time()
    current_sequence = list_of_candidates[i]
    print(f"Processing Candidate #{i}: {current_sequence}")
    # candidate string must contain all of the permutations
    permutations_remaining = permutations_list.copy()
    permutations_remaining.remove(current_sequence)
    print(f"{len(permutations_remaining)} permutations to add")
    while len(permutations_remaining) > 0:
        if len(permutations_remaining) % 1000 == 0:
#             print(f"    current string length: {len(current_sequence)}")
            print(f"    Permutations remaining... {len(permutations_remaining)}")
        best_permutation = permutations_remaining[0]
        best_added_length = 7
        for permutation in permutations_remaining:
            # check if already in sequence
            if operator.contains(current_sequence, permutation):
                permutations_remaining.remove(permutation)
                continue
            added_length = get_added_length(current_sequence, permutation)
            if added_length < best_added_length:
                best_added_length = added_length
                best_permutation = permutation
        current_sequence = (current_sequence + best_permutation[-best_added_length:])
        permutations_remaining.remove(best_permutation)
    list_of_candidates[i] = current_sequence
    print(f"    Candidate processed!")
    print(f"    Total Permutation length: {len(current_sequence)}")
    # Test that all substrings are part of the lists
    print(f"    All permutations are included: {all(substring in current_sequence for substring in permutations_list)}")
    end_candidate_sequence = time.time()
    print(f"Time to process candidate: {round(end_candidate_sequence - start_candidate_sequence)} seconds\n")
end = time.time()
print(f"Total time to process 20 candidates: {round(end - start)/60} minutes")

Processing Candidate #2215: 🧝🎅🎄🦌🤶🎀🎁
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6247
    All permutations are included: False
Time to process candidate: 112 seconds

Processing Candidate #2308: 🧝🤶🦌🎅🎀🎄🎁
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6260
    All permutations are included: False
Time to process candidate: 115 seconds

Processing Candidate #2464: 🧝🦌🎄🎁🎀🎅🤶
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candid

    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6278
    All permutations are included: False
Time to process candidate: 110 seconds

Processing Candidate #1892: 🦌🎄🎁🎀🤶🎅🧝
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6251
    All permutations are included: False
Time to process candidate: 111 seconds

Processing Candidate #2312: 🧝🤶🦌🎄🎁🎅🎀
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6262
    All permutations are included: False
Time to process candidate: 109 seconds



    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6291
    All permutations are included: False
Time to process candidate: 1949 seconds

Processing Candidate #881: 🤶🦌🧝🎁🎀🎄🎅
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6253
    All permutations are included: False
Time to process candidate: 2404 seconds

Processing Candidate #669: 🎅🎀🧝🎁🦌🎄🤶
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6283
    All per

In [ ]:
# the shortest list of these 20 candidates is 5,913
# but it's only 1/6 of our potential candidates for submission
len(min(list_of_candidates[:20], key=len))

In [ ]:
# indexes 0, 6, and 8 have thes shortest strings
# Our score will be the largest, 5,950
results = [(len(candidate), list_of_candidates.index(candidate)) for candidate in list_of_candidates[:20]]
results.sort()
print(results)

An improvement: add to the beginning or the end of the permutation

In [59]:
all_permutations = permutations.values.tolist()
all_permutations = [item for sublist in all_permutations for item in sublist]

In [ ]:
list_of_candidates = permutations_list.copy()
# For each potential starting string generate a candidate output
start = time.time()
for i in range(0, 50):
    start_candidate_sequence = time.time()
    current_sequence = list_of_candidates[i]
    print(f"Processing Candidate #{i}: {current_sequence}")
    # candidate string must contain all of the permutations
    permutations_remaining = all_permutations.copy()
    permutations_remaining.remove(current_sequence)
    print(f"{len(permutations_remaining)} permutations to add")
    while len(permutations_remaining) > 0:
        if len(permutations_remaining) % 1000 == 0:
#             print(f"    current string length: {len(current_sequence)}")
            print(f"    Permutations remaining... {len(permutations_remaining)}")
        best_permutation = permutations_remaining[0]
        best_added_length = 7
        for permutation in permutations_remaining:
            # check if already in sequence
            if operator.contains(current_sequence, permutation):
                permutations_remaining.remove(permutation)
                continue
            added_length = get_added_length(current_sequence, permutation)
            if added_length < best_added_length:
                best_added_length = added_length
                best_permutation = permutation
        current_sequence = (current_sequence + best_permutation[-best_added_length:])
        permutations_remaining.remove(best_permutation)
    list_of_candidates[i] = current_sequence
    print(f"    Candidate processed!")
    print(f"    Total Permutation length: {len(current_sequence)}")
    # Test that all substrings are part of the lists
    print(f"    All permutations are included: {all(substring in current_sequence for substring in permutations_list)}")
    end_candidate_sequence = time.time()
    print(f"Time to process candidate: {round(end_candidate_sequence - start_candidate_sequence)} seconds\n")
end = time.time()
print(f"Total time to process 20 candidates: {round(end - start)/60} minutes")

Processing Candidate #2215: 🧝🎅🎄🦌🤶🎀🎁
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6247
    All permutations are included: False
Time to process candidate: 112 seconds

Processing Candidate #2308: 🧝🤶🦌🎅🎀🎄🎁
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candidate processed!
    Total Permutation length: 6260
    All permutations are included: False
Time to process candidate: 115 seconds

Processing Candidate #2464: 🧝🦌🎄🎁🎀🎅🤶
5039 permutations to add
    Permutations remaining... 5000
    Permutations remaining... 4000
    Permutations remaining... 3000
    Permutations remaining... 2000
    Permutations remaining... 1000
    Candid

# Limiting the Search Space

It would take a very long time to brute force this search over all candidates. This approach is not efficient or scalable. What if we had 5,000,000 instead of 5,040 permutations? What if the if the first two positions were not fixed and we had to search over all of the permutations? What if we incorporated wild cards? Can we come up with a way to search this space in a greedy manner?

In [ ]:
# Start with a an initial permutation.
# Add to the beginning or the end of the permutation whichever

## Using the Distance Matrix

What are ways in which we can reduce the search space in our brute force approach?
1. Order searches by using the distance matrix
2. 

# Baseline Submission
Let's generate a submission that's above the sample submission. This will serve as our new baseline

In [61]:
baseline_submission = pd.DataFrame(data={"schedule": [list_of_candidates[0], part_a_candidates[0], part_a_candidates[1]]})

In [62]:
baseline_submission.head()

,schedule
0,🎅🤶🦌🧝🎄🎁🎀
1,🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🧝🎀🎄🎁🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🎄🧝🎁🎀🎅🤶🦌🎄🧝🎀🎁🎅🤶🦌🎄...
2,🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎀🎄🎁🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🎄🧝🎁🎀🎅🤶🦌🎄🧝🎀🎁🎅🤶🦌🎄...


In [ ]:
baseline_submission.to_csv("submission.csv", sep=',',index=False)

# Recursion

But first, can we simplify our method with recursion?

# Parallel Processing

# With Wildcards
TODO: add the beggingin wild cards to the starting permutations

# Shortest Common Super String